# Re-Ranking for Topic Diversification

Recommender systems (RecSys) are a class of machine learning models built to recommend the most relevant things to the user - an ad that a customer is most likely to click on, a product that a customer is most likely to buy, and so on. In doing so, RecSys often exploit known preferences of the users to ensure that suggestions from it are relevant. This leads to an of echo chamber of sorts, for example, if you were to purchase a summer dress from a retail website, you might be shown other summer dresses everytime you visit the website because the RecSys knows that you were interested in summer dresses in the past.

Understandably, this leads to some obvious issues like:
- bad user experience from constantly seeing a product that might not be relevant to them anymore
- not exposing other products from the catalog that user might have bought leading to lost revenues for the company
aling with a number of nuanced issues.

One can think of a few ways to resolve these issues like:
- Ensure that the output of the model recommends a diverse list of topics by changes to features or type of model
- Re-ranking the items recommended by the RecSys to ensure it is diverse

For the purpose of this exercise, we will look at the latter, the problem of re-ranking. We have a dataset of movie ratings ([MovieLens Dataset](https://grouplens.org/datasets/movielens/1m/ "MovieLens 1M dataset")), a stable benchmark dataset for recommender systems. It has 1 million ratings from 6000 users on 4000 movies (or 4.16% user-movie interactions covered).

## Sections
1. [Methodology](#Methodology) 
2. [Building a baseline RecSys](#Building-a-baseline-RecSys)
3. [Diversification as a Set Cover Problem // A Greedy Solution](#Diversification-as-a-Set-Cover-Problem-//-A-Greed-Solution)
4. [Diversification as a Set Cover Problem // A Mixed Integer Programming Solution](#Diversification-as-a-Set-Cover-Problem-//-A-Mixed-Integer-Programming-Solution)


## Methodology
We will build two recommender systems:  
1. **Baseline** - Recommend 10 top-rated movies from this dataset.  
2. **Version 1** - Diversified recommender system using a greedy algorithm
3. **Version 2** - Diversified recommender system using a mixed linear programming approach

Reference:
- The problem is inspired by the blog here: https://nbviewer.jupyter.org/github/david-cortes/datascienceprojects/blob/master/machine_learning/topic_diversification.ipynb


## Building a baseline RecSys

In [39]:
# Importing data
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('data/ml-1m/movies.dat', sep='::', engine='python')
ratings = pd.read_csv('data/ml-1m/ratings.dat', sep='::', engine='python')
users = pd.read_csv('data/ml-1m/users.dat', sep='::', engine='python')

In [40]:
print(f"Data in movies = {list(movies.columns)}")
print(f"Data in ratings = {list(ratings.columns)}")
print(f"Data in users = {list(users.columns)}")

Data in movies = ['MovieID', 'Title', 'Genres']
Data in ratings = ['UserID', 'MovieID', 'Rating', 'Timestamp']
Data in users = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']


In [41]:
# Getting the top-rated movies
import numpy as np

topRated = ratings.assign(raters=ratings.Rating>=0)\
    .groupby('MovieID')\
    .agg({'Rating': 'mean','UserID': 'count'})\
    .rename(columns={'UserID': 'UserCount', 'Rating': 'AvgRating'})\
    .assign(score=lambda x: x.AvgRating*x.UserCount)\
    .sort_values('score',ascending=False)\
    .head(50)

topRated.join(movies.set_index('MovieID')).head(10)

,AvgRating,UserCount,score,Title,Genres
MovieID,,,,,
2858,4.317386,3428,14800.0,American Beauty (1999),Comedy|Drama
260,4.453694,2991,13321.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1196,4.292977,2990,12836.0,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1210,4.022893,2883,11598.0,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2028,4.337354,2653,11507.0,Saving Private Ryan (1998),Action|Drama|War
1198,4.477725,2514,11257.0,Raiders of the Lost Ark (1981),Action|Adventure
593,4.351823,2578,11219.0,"Silence of the Lambs, The (1991)",Drama|Thriller
2571,4.315830,2590,11178.0,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2762,4.406263,2459,10835.0,"Sixth Sense, The (1999)",Thriller


**Observation** - Only 8 of 18 genres are present in the top 10 movies that have been recommended by the RecSys. Likely that users have a preference for many types of movies, and using this recommendation might not satisfy every user.

## Assessing the diversity of the solution
We want to understand what the universe looks like, and how do we measure the diversity in the context of genres

In [42]:
def get_unique_items_in_column(dataset, colname):
    """Finds unique items in a column
    For a given Pandas DataFrame, the function finds all the unique
    items in the column. The function expects the column to contain
    either one value or a list of values.

    Args:
        dataset (obj): Pandas DataFrame with data
        colname (str): Column that needs to be scanned

    Returns:
        obj : A set object with unique items
    """
    uniqueItems = set()

    for index, data in dataset.iterrows():
        if isinstance(data[colname], list):
            uniqueItems.update(data[colname])
        else:
            uniqueItems.add(data[colname])
    return uniqueItems

movies['Genres'] = movies['Genres'].apply(lambda x : x.strip().split('|'))
genresUniverse = get_unique_items_in_column(movies, 'Genres')
print(f"Number of genres in the dataset = {len(genresUniverse)}")

Number of genres in the dataset = 18


## Diversification as a Set Cover Problem // A Greedy Solution
Now that we have a list of top movies, one way to introduce more genres, can be to reshuffle the movies in the top 50 so that all genres are represented in the top 10 recommendations. In doing so, we can use the maximum coverage formulation, where movies that have more coverage across genres, are represented and give us a better chance of getting more genres in.

For this exercise, we are going to use the greedy algorithm for set cover problems. This is a common algorithm is computer science, and it's a polynomial time approximation of the NP-hard problem of finding a subset of items that covers all elements in the universe. 

In [54]:
# Finding the movies to rank
moviesToRerank = topRated.join(movies.set_index('MovieID'))
# Finding the genres that are covered in the top N movies, so that we can inlude all genres
genresInList = get_unique_items_in_column(moviesToRerank, 'Genres')

# Setting constants, and initializing variables
uncoveredGenres = set()
movieIndex = 0
recListSize = 10
reRankedMovies = pd.DataFrame()

while genresInList.difference(uncoveredGenres) and movieIndex < recListSize: 

    # Get the number of genres covered by the movies
    moviesToRerank['nGenres'] = moviesToRerank['Genres'].apply(lambda x: len(set(x).difference(uncoveredGenres)) )
    # Order movies by descending order or Genres covered in dataset
    moviesToRerank = moviesToRerank.sort_values('nGenres', ascending=False)
    moviesToRerank = moviesToRerank.reset_index(drop=True)

    # Adding the top movie to the list
    topMovie = moviesToRerank.iloc[0]
    reRankedMovies = reRankedMovies.append(topMovie)
    uncoveredGenres.update(topMovie['Genres'])
    movieIndex += 1

    # Dropping the row
    moviesToRerank = moviesToRerank.drop([0])
    
# Add movies to the list, if we don't have a full list yet
moviesToRerank = moviesToRerank.sort_values('score', ascending=False)
moviesToRerank = moviesToRerank.reset_index(drop=True)

for index, movie in moviesToRerank.iterrows():
    if movieIndex >= recListSize:
        break
    elif movieIndex < recListSize and movie.Title not in list(reRankedMovies.Title):
        reRankedMovies = reRankedMovies.append(movie)
        movieIndex += 1


In [56]:
reRankedMovies

,AvgRating,Genres,Title,UserCount,nGenres,score
0,4.292977,"[Action, Adventure, Drama, Sci-Fi, War]",Star Wars: Episode V - The Empire Strikes Back...,2990.0,5.0,12836.0
0,4.219406,"[Crime, Film-Noir, Mystery, Thriller]",L.A. Confidential (1997),2288.0,4.0,9654.0
0,4.146846,"[Animation, Children's, Comedy]",Toy Story (1995),2077.0,3.0,8613.0
0,3.953029,"[Comedy, Romance]",Groundhog Day (1993),2278.0,1.0,9005.0
0,3.409778,"[Action, Adventure, Fantasy, Sci-Fi]",Star Wars: Episode I - The Phantom Menace (1999),2250.0,1.0,7672.0
0,4.159585,"[Action, Horror, Sci-Fi, Thriller]",Alien (1979),2024.0,1.0,8419.0
0,4.247963,"[Adventure, Children's, Drama, Musical]","Wizard of Oz, The (1939)",1718.0,1.0,7298.0
0,4.317386,"[Comedy, Drama]",American Beauty (1999),3428.0,0.0,14800.0
1,4.453694,"[Action, Adventure, Fantasy, Sci-Fi]",Star Wars: Episode IV - A New Hope (1977),2991.0,0.0,13321.0
2,4.022893,"[Action, Adventure, Romance, Sci-Fi, War]",Star Wars: Episode VI - Return of the Jedi (1983),2883.0,0.0,11598.0


## Diversification as a Set Cover Problem // A Mixed Integer Programming Solution
Another way of solving the problem above comes from the field of operations research. We can used mixed integer programming to 

In [ ]:
b